In [ ]:
# 完整脚本：使用本地 MiDaS 源码和权重进行深度估计和雾图合成
import os
import sys
import cv2
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm

# 添加本地 MiDaS 源码路径
sys.path.append(os.path.abspath("MiDaS"))
from midas.dpt_depth import DPTDepthModel
from midas.transforms import Resize, NormalizeImage, PrepareForNet
from torchvision.transforms import Compose

# 路径配置
input_dir = Path("autodl-tmp/afo/images1")              # 输入图像
output_dir = Path("autodl-tmp/afo_fog/images_beta=3")         # 输出加雾图像
model_path = Path("MiDaS/weights/dpt_hybrid_384.pt")    # 本地模型权重

output_dir.mkdir(parents=True, exist_ok=True)

# 模型配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas = DPTDepthModel(
    path=str(model_path),
    backbone="vitb_rn50_384",  # 对应 hybrid 模型结构
    non_negative=True
).to(device)
midas.eval()

# 图像预处理流程（本地）
transform = Compose([
    Resize(
        384, 384,
        resize_target=None,
        keep_aspect_ratio=True,
        ensure_multiple_of=32,
        resize_method="minimal",
        image_interpolation_method=cv2.INTER_CUBIC,
    ),
    NormalizeImage(mean=[0.485, 0.456, 0.406],
                   std=[0.229, 0.224, 0.225]),
    PrepareForNet()
])

# 雾气参数
fog_beta = 3.0  # 控制雾浓度（0.05~0.15）
fog_color = 255  # 大气光颜色

# 辅助函数
def depth_to_transmission(depth, beta=0.08):
    depth = cv2.normalize(depth, None, 0, 1, cv2.NORM_MINMAX)
    t = np.exp(-beta * depth)
    return np.clip(t, 0.05, 1.0)

def apply_fog(img, transmission, A=255):
    fog = img * transmission[:, :, None] + A * (1 - transmission[:, :, None])
    return np.clip(fog, 0, 255).astype(np.uint8)

# 主循环
img_list = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png'))])
with tqdm(total=len(img_list), desc='处理加雾图') as pbar:
    for fname in img_list:
        img_path = input_dir / fname
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 深度预测
        input_tensor = transform({"image": img_rgb})["image"]
        input_tensor = torch.from_numpy(input_tensor).unsqueeze(0).to(device)
        with torch.no_grad():
            prediction = midas(input_tensor)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img_rgb.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze().cpu().numpy()

        # 合成雾图
        t = depth_to_transmission(prediction, beta=fog_beta)
        fog_img = apply_fog(img, t, A=fog_color)
        cv2.imwrite(str(output_dir / fname), fog_img)

        pbar.update(1)

"✅ 所有图像处理完成，加雾图已保存至：" + str(output_dir)


处理加雾图:   0%|          | 0/10 [00:00<?, ?it/s]